In [137]:
# !pip install ipycanvas

In [2]:
# install ipycanvas if not already
# !pip install ipycanvas

from ipycanvas import Canvas, hold_canvas
from IPython.display import display
import math
import random
import time

# ---------------------------
# Node and Edge data classes
# ---------------------------
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.vx = 0.0
        self.vy = 0.0
        self.radius = 6
        self.friction = 0.9  # initial friction

class Edge:
    def __init__(self, a, b, rest_length):
        self.a = a
        self.b = b
        self.rest = rest_length
        self.phase = 0.0      # contraction phase [0,1]
        self.contracting = False

# ---------------------------
# Playground / Physics
# ---------------------------
class Creature:
    def __init__(self, canvas_width=1200, canvas_height=600):
        self.width = canvas_width
        self.height = canvas_height
        self.nodes = []
        self.edges = []
        self.trail = []
        self.max_trail = 20000
        
        # physics constants
        self.dt = 0.6
        self.global_damping = 0.995
        self.base_friction_high = 0.9
        self.base_friction_low = 0.1
        self.contraction_alpha = 0.25
        self.contraction_speed = 0.02
        self.lift_factor = 0.9
        self.spring_k = 0.15
        
        # canvas
        self.canvas = Canvas(width=self.width, height=self.height)
        display(self.canvas)
    
    # ---------------------------
    # Graph initialization
    # ---------------------------
    def build_connected_graph(self, N=10, extra_edges=9):
        self.nodes = []
        self.edges = []
        for i in range(N):
            angle = (i / N) * 2*math.pi + (random.random()-0.5)*0.5
            r = 80 + random.random()*60
            cx = self.width/2 + math.cos(angle)*r
            cy = self.height/2 + math.sin(angle)*r
            self.nodes.append(Node(cx + (random.random()-0.5)*40, cy + (random.random()-0.5)*40))
        # ensure connectivity
        for i in range(1,N):
            j = random.randint(0,i-1)
            rest_len = self.distance(self.nodes[i], self.nodes[j])
            self.edges.append(Edge(i,j, rest_len))
        # add extra random edges
        tries=0
        while len(self.edges) < (N-1+extra_edges) and tries<200:
            a = random.randint(0,N-1)
            b = random.randint(0,N-1)
            if a!=b and not self.edge_exists(a,b):
                rest_len = self.distance(self.nodes[a], self.nodes[b])
                self.edges.append(Edge(a,b,rest_len))
            tries += 1
    
    def edge_exists(self,a,b):
        return any((e.a==a and e.b==b) or (e.a==b and e.b==a) for e in self.edges)
    
    # ---------------------------
    # Physics update
    # ---------------------------
    def step_physics(self):
        # update contraction phases
        for e in self.edges:
            if e.contracting:
                e.phase += self.contraction_speed
                if e.phase >= 1.0:
                    e.phase = 1.0
                    e.contracting = False
            else:
                e.phase -= self.contraction_speed
                if e.phase < 0.0:
                    e.phase = 0.0
        
        # update node friction based on oriented contractions
        for idx, n in enumerate(self.nodes):
            max_phase = 0.0
            for e in self.edges:
                # if this node is the lifted one
                if e.phase>0 and hasattr(e,'lifted') and e.lifted==idx:
                    if e.phase>max_phase: max_phase = e.phase
            n.friction = self.base_friction_high*(1 - self.lift_factor*max_phase) + self.base_friction_low*(self.lift_factor*max_phase)
        
        # spring forces
        for e in self.edges:
            n1 = self.nodes[e.a]
            n2 = self.nodes[e.b]
            dx = n2.x - n1.x
            dy = n2.y - n1.y
            d = math.sqrt(dx*dx + dy*dy)+1e-8
            L = e.rest*(1 - self.contraction_alpha*e.phase)
            F = self.spring_k*(d-L)
            fx = F*dx/d
            fy = F*dy/d
            n1.vx += fx*0.5
            n1.vy += fy*0.5
            n2.vx -= fx*0.5
            n2.vy -= fy*0.5
        
        # update node positions
        for n in self.nodes:
            n.vx *= n.friction
            n.vy *= n.friction
            n.vx *= self.global_damping
            n.vy *= self.global_damping
            n.x += n.vx*self.dt
            n.y += n.vy*self.dt
        
        # update trail
        cx = sum(n.x for n in self.nodes)/len(self.nodes)
        cy = sum(n.y for n in self.nodes)/len(self.nodes)
        self.trail.append((cx,cy))
        if len(self.trail)>self.max_trail:
            self.trail.pop(0)
    
    # ---------------------------
    # Graph control API
    # ---------------------------
    def grow_node(self, parent_idx):
        parent = self.nodes[parent_idx]
        angle = random.random()*2*math.pi
        r = 30+random.random()*20
        x = parent.x + math.cos(angle)*r
        y = parent.y + math.sin(angle)*r
        new_node = Node(x,y)
        new_idx = len(self.nodes)
        self.nodes.append(new_node)
        rest_len = self.distance(parent, new_node)
        self.edges.append(Edge(parent_idx, new_idx, rest_len))
        return new_idx
    
    def grow_edge(self, a,b):
        if a!=b and not self.edge_exists(a,b):
            rest_len = self.distance(self.nodes[a], self.nodes[b])
            self.edges.append(Edge(a,b,rest_len))
    
    def contract_edge(self,a,b):
        for e in self.edges:
            if e.a==a and e.b==b:
                e.contracting = True
                e.phase = 0.0
                e.lifted = b  # oriented contraction: b is lifted
                break
    
    def distance(self,n1,n2):
        return math.sqrt((n1.x-n2.x)**2 + (n1.y-n2.y)**2)
    
    # ---------------------------
    # Drawing
    # ---------------------------
    def draw(self):
        c = self.canvas
        c.clear()
        
        # trail
        c.stroke_style = "white"
        c.begin_path()
        if self.trail:
            c.move_to(*self.trail[0])
            for x,y in self.trail[1:]:
                c.line_to(x,y)
            c.stroke()
        
        # edges
        for e in self.edges:
            n1 = self.nodes[e.a]
            n2 = self.nodes[e.b]
            c.stroke_style = "red" if e.phase>0 else "gray"
            c.line_width = 2
            c.begin_path()
            c.move_to(n1.x,n1.y)
            c.line_to(n2.x,n2.y)
            c.stroke()
        
        # nodes
        for n in self.nodes:
            f = (n.friction - self.base_friction_low)/(self.base_friction_high-self.base_friction_low)
            f = max(0, min(1,f))
            r = 30*(1-f)+20*f
            g = 200*f+80*(1-f)
            b = 200*(1-f)+120*f
            c.fill_style = f"rgb({r},{g},{b})"
            c.begin_path()
            c.arc(n.x,n.y,n.radius,0,2*math.pi)
            c.fill()
            # friction bar
            c.fill_style="#222"
            c.fill_rect(n.x-10,n.y+10,20,4)
            c.fill_style="#fff"
            c.fill_rect(n.x-10,n.y+10,20*f,4)
        
        # center of mass
        cx = sum(n.x for n in self.nodes)/len(self.nodes)
        cy = sum(n.y for n in self.nodes)/len(self.nodes)
        c.fill_style="white"
        c.begin_path()
        c.arc(cx,cy,3,0,2*math.pi)
        c.fill()
    
    # ---------------------------
    # Single frame update
    # ---------------------------
    def step(self):
        self.step_physics()
        self.draw()


In [4]:
creature = Creature()
creature.build_connected_graph(N=10)

import asyncio

async def animate(creature, fps=60):
    dt = 1/fps
    while True:
        with hold_canvas(creature.canvas):
            creature.step()
        await asyncio.sleep(dt)

# start animation
asyncio.ensure_future(animate(creature))




Canvas(height=600, width=1200)

<Task pending name='Task-5' coro=<animate() running at /tmp/ipykernel_2444790/878248456.py:6>>

In [6]:
# grow a new node connected to node 0
new_idx = creature.grow_node(0)

# grow an edge between node 1 and 2
creature.grow_edge(1,2)

# contract edge from 0->1 (0 stays, 1 lifts)
creature.contract_edge(0,1)
